In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer ( ) 

import numpy 
import tflearn 
import tensorflow
import random

import json
with open('/Users/emrekaya/Desktop/intents_AKUT.json') as file:
    data = json. load(file)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
words = []
labels = []
docs_x = []
docs_y = []

In [3]:
for intent in data['intents']:
    for pattern in intent ['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append (wrds)
        docs_y.append(intent["tag"]) 
    if intent[ 'tag'] not in labels:
        labels.append(intent ['tag'])

In [4]:
words = [stemmer.stem(w.lower ( )) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted (labels)

In [5]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate (docs_x):
    bag = []
    
    wrds = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append (0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append (bag)
    output.append (output_row)

In [6]:
training = numpy.array(training)
output = numpy.array (output)

In [7]:

tensorflow.compat.v1.reset_default_graph()
net = tflearn.input_data(shape= [None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected (net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression (net)
model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Metal device set to: Apple M1 Max


2022-11-19 20:34:58.260690: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-19 20:34:58.260844: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-11-19 20:34:58.344789: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-19 20:34:58.346561: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-19 20:34:58.346707: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-19 20:34:58.350303: I tensorflow/core/grappler/optimizers/custom_graph

In [8]:
model.fit(training, output, n_epoch=1500, batch_size=8, show_metric=True)
model.save ("model.tflearn")


Training Step: 4499  | total loss: 0.00378 | time: 0.042s
| Adam | epoch: 1500 | loss: 0.00378 - acc: 1.0000 -- iter: 16/20
Training Step: 4500  | total loss: 0.00353 | time: 0.062s
| Adam | epoch: 1500 | loss: 0.00353 - acc: 1.0000 -- iter: 20/20
--
INFO:tensorflow:/Users/emrekaya/tensorflow-test/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/Users/emrekaya/tensorflow-test/model.tflearn.meta
INFO:tensorflow:100
INFO:tensorflow:/Users/emrekaya/tensorflow-test/model.tflearn.index
INFO:tensorflow:100
INFO:tensorflow:/Users/emrekaya/tensorflow-test/model.tflearn.data-00000-of-00001
INFO:tensorflow:100


2022-11-19 20:36:35.373572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se: bag[i] = 1

    return numpy.array(bag)
    
def chat ( ): 
    print ("Start talking with your bot (type quit to stop)!")
    while True:
        inp = input ( "You: ")
        if inp.lower () == "quit":
            break
        results = model.predict([bag_of_words(inp, words) ])
        results_index = numpy.argmax(results)
        tag = labels[results_index] 
    
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        print(random.choice (responses))
chat()

Start talking with your bot (type quit to stop)!
You: üye olabilir miyim


2022-11-19 20:36:45.688590: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-19 20:36:45.699854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Memnunum
You: gönüllü olmak isitiyorum
Yardım etmekten mutluluk duyarım!
You: nasıl akut üyesei olurum
Memnunum
